># Universidad Autónoma de Aguascalientes
>## *Ingeniería en Computación Inteligente*
>#### Materia:
>Inteligencia Artificial
>#### Topico: 
>Proyecto Final:
>- _Reconocimiento de Expresiones con red neuronal._
>#### Integrantes del Equipo:
>- Juan Francisco Gallo Ramírez
>- José Alfredo Díaz Robledo
>- Luis Palbo Esparza Terrones
>- Luis Manuel Flores Jiménez 
>#### Maestro: 
>Dr. Francisco Javier Luna Rosas
>#### Fecha: 
>Noviembre del 2023

# Proyecto Final: _Reconocimiento de Expresiones con Red Neuronal._

## ▪️ Se importan la librerías correspondientes.

In [2]:
import cv2
import math
import mediapipe as mp
import numpy as np
import openpyxl
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier

## ▪️ Se define la funcción para calcular distancias.

Se define la función que no ayuda a calcular la distancia entre los puntos faciales, esto para realizar el dataset que no ayudará a detectar las expresiones.

In [3]:
def calcular_distancia(punto1, punto2):
    x1, y1, z1 = punto1.x, punto1.y, punto1.z
    x2, y2, z2 = punto2.x, punto2.y, punto2.z
    distancia = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)
    return distancia

## ▪️ Se crea el archivo excel y se nombran las celdas.

El archivo excel es el que almacenará nuestro dataset de las distanticias entre los puntos faciales.

In [3]:
wb = openpyxl.Workbook()
sheet = wb.active
sheet['A1'] = "Distancia entre Ceja y Ojo";
sheet['B1'] = "Apertura de Ojo";
sheet['C1'] = "Ancho de Boca";  
sheet['D1'] = "Apertura de Boca";
sheet['E1'] = "Distancia de Extremos y Superior Boca";
sheet['F1'] = "Distancia de Extremos e Inferior Boca";
sheet['G1'] = "EXPRESIÓN"

## ▪️ Se define la función para obtener los datos faciales.

Esta función recibe como parametros la expresión, que es el clasificador que se colocará en la última celda; un booleano para establecer la preferencia de mostrar la máscara de puntos o no; la cantidad, que es la cantidad de fotogramas que se procesarán para el dataset; y finalmente el archivo excel donde se guardará el dataset.

In [4]:
def fillDataSet(expresion, show_mask, cantidad, excel):
    i = 0
    showMask = show_mask
    mp_face_mesh = mp.solutions.face_mesh
    mp_drawing = mp.solutions.drawing_utils

    # Inicializar la cámara.
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error al abrir la cámara.")
        exit()
        
    # Crear un objeto CascadeClassifier para la detección de rostros.
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Verificar si la cámara se abrió correctamente.
    with mp_face_mesh.FaceMesh(static_image_mode=False,
                               max_num_faces=1,
                               min_detection_confidence=0.5) as face_mesh:
        
        # Se capturarán los datos según la cantidad indicada.
        while i < cantidad:
            # Capturar un solo fotograma.
            ret, frame = cap.read()

            # Verificar si la lectura del fotograma fue exitosa
            if not ret:
                print("Error al leer el fotograma.")
                break

            # Transformar el frame en escala de grises para una mejor detección.
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            # Realizar la detección de rostros.
            faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))
            
            for (x, y, w, h) in faces:
                
                # Dibujar un rectángulo alrededor del rostro.
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 220), 2)
                # Dibujar texto por encima del cuadro.
                cv2.putText(frame, "Entrenando expresion:" , (x, y-30), cv2.FONT_HERSHEY_PLAIN , 0.8, (0, 255, 220), 2)
                cv2.putText(frame, expresion , (x, y-10), cv2.FONT_HERSHEY_SIMPLEX , 0.8, (255, 255, 255), 2)

                    
                # Recortar y guardar el rectángulo del rostro
                face_roi = frame[y:y+h, x:x+w]

                # Se aplica la mascara de punto facemesh.
                frame_rgb = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
                results = face_mesh.process(frame_rgb)

                if results.multi_face_landmarks is not None:
                    for face_landmarks in results.multi_face_landmarks:

                        #>>> PUNTOS DE BOCA >>>>>>>>>
                        # Punto medio labio superior.
                        p13 = face_landmarks.landmark[13]
                        # Punto medio labio inferior.
                        p14 = face_landmarks.landmark[14]
                        # Punto extremo izquierdo.
                        p61 = face_landmarks.landmark[61]
                        # Punto extremo derecho.
                        p306 = face_landmarks.landmark[306]

                        #>>> PUNTOS DE CEJA >>>>>>>>>
                        # Ceja izquierda.
                        p65 = face_landmarks.landmark[65]
                        # Ceja derecha.
                        p295 = face_landmarks.landmark[295]

                        #>>> PUNTOS DE OJO >>>>>>>>>
                        # Ojo izquierdo ----
                        # Punto superior.
                        p159 = face_landmarks.landmark[159]
                        # Punto inferior.
                        p145 = face_landmarks.landmark[145]
                        # Ojo derecho ------
                        # Punto superior.
                        p386 = face_landmarks.landmark[386]
                        # Punto inferior.
                        p374 = face_landmarks.landmark[374]
                        sheet = wb.active

                        # === Se hacen los respectivos cálculos de distancias entre puntos.
                        # Distancia entre Ceja y Ojo.
                        d1 = (calcular_distancia(p65, p159) + calcular_distancia(p295, p386)) / 2
                        # Apertura de Ojo".
                        d2 = (calcular_distancia(p159, p145) + calcular_distancia(p386, p374)) / 2
                        # Ancho de Boca.
                        d3 = calcular_distancia(p61, p306)
                        # Apertura de Boca.
                        d4 = calcular_distancia(p13, p14)
                        # Distancia de Extremos y Superior Boca.
                        d5 = (calcular_distancia(p61, p13) + calcular_distancia(p306, p13)) / 2
                        # Distancia de Extremos e Inferior Boca.
                        d6 = (calcular_distancia(p61, p14) + calcular_distancia(p306, p14)) / 2
                        # Clasificador.
                        d7 = expresion

                        if showMask:
                        # Dibujar la malla facial en el fotograma recortado
                            mp_drawing.draw_landmarks(
                                face_roi, face_landmarks, mp_face_mesh.FACEMESH_CONTOURS,
                                landmark_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1, circle_radius=1),
                                connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=1, circle_radius=1)
                            )
                            
                        i = i + 1
                        # Se agregan los datos al dataset.
                        sheet.append([d1, d2, d3, d4, d5, d6, d7])

            # Mostrar el fotograma con los rostros detectados.
            cv2.imshow('Sistema de Reconocimiento de Exrpesiones', frame)

            # Salir del bucle si se presiona la tecla 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Liberar la cámara y cerrar la ventana.
    cap.release()
    cv2.destroyAllWindows()
    # Guardar en el archivo.
    excel.save('DataSet.xlsx')

### Se llena el dataset con expresión: _Neutral_ 😐

In [5]:
fillDataSet("Neutral", True, 700, wb)

### Se llena el dataset con expresión: _Feliz_ 😁

In [6]:
fillDataSet("Feliz", True, 700, wb)

### Se llena el dataset con expresión: _Triste_ 🙁

In [7]:
fillDataSet("Triste", True, 700, wb)

### Se llena el dataset con expresión: _Enojado_ 😠

In [8]:
fillDataSet("Enojado", True, 700, wb)

### Se llena el dataset con expresión: _Sorprendido_ 😮

In [9]:
fillDataSet("Sorprendido", True, 700, wb)

## ▪️ Red Neuronal

Se lee el archivo del dataset, y se muestran la respectiuva matriz categórica y matriz a predecir.

In [9]:
data = pd.read_excel('DataSet.xlsx')
X = data.iloc[0:,0:6]
y = data.iloc[:,6]
X = np.array(X)
y = np.array(y)
print( ">> Se muestra la matriz categorica:\n")
print(X)
print( "\n>> Se muestra la matriz a predecir:\n")
print(y)

>> Se muestra la matriz categorica:

[[0.10600105 0.05144231 0.26750478 0.01123014 0.15149762 0.14904705]
 [0.10756348 0.05149    0.28032289 0.01356024 0.15962655 0.15604199]
 [0.10639915 0.0484678  0.27683848 0.01160979 0.15743412 0.15481244]
 ...
 [0.12328187 0.05783282 0.29886252 0.12753206 0.18379155 0.17349823]
 [0.12418599 0.05882374 0.29708943 0.13052481 0.18329715 0.17251083]
 [0.12737746 0.05845203 0.29758004 0.10806184 0.17942204 0.16710349]]

>> Se muestra la matriz a predecir:

['Neutral' 'Neutral' 'Neutral' ... 'Sorprendido' 'Sorprendido'
 'Sorprendido']


Se separan los datos con el 70% de los datos para entrenamiento y el 30% para testing:

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=0)

Mediante el constructor inicializa la instancia_red.

In [11]:
instancia_red = MLPClassifier(solver='lbfgs', random_state=0, max_iter=2000)
print(instancia_red)
instancia_red.fit(X_train,y_train)

MLPClassifier(max_iter=2000, random_state=0, solver='lbfgs')


MLPClassifier(max_iter=2000, random_state=0, solver='lbfgs')

Las predicciones del testing se muestran:

In [12]:
print(">> Las predicciones en Testing son: {}".format(instancia_red.predict(X_test)))

>> Las predicciones en Testing son: ['Feliz' 'Neutral' 'Sorprendido' ... 'Sorprendido' 'Feliz' 'Neutral']


Función para calcular los índices de calidad de la predicción:

In [13]:
def indices_general(MC, nombres = None):
    precision_global = np.sum(MC.diagonal()) / np.sum(MC)
    error_global = 1 - precision_global
    return {">> Matriz de Confusión":MC, 
            ">> Precisión Global":precision_global, 
            ">> Error Global":error_global}

## ▪️ Índices de calidad del modelo.

In [14]:
prediccion = instancia_red.predict(X_test)
MC = confusion_matrix(y_test, prediccion)
indices = indices_general(MC,list(np.unique(y)))
for k in indices:
    print("\n%s:\n%s"%(k,str(indices[k])))


>> Matriz de Confusión:
[[206   0   0   0   0]
 [  0 210   0   0   0]
 [  0   0 223   0   0]
 [  0   0   0 216   0]
 [  0   0   0   0 195]]

>> Precisión Global:
1.0

>> Error Global:
0.0


Diccionario para texto y color según la predicción.

In [15]:
expresion = {
    "['Neutral']" : ["Neutral", (123, 125, 125)],
    "['Feliz']" : ["Feliz", (15, 196, 241)],
    "['Triste']" : ["Triste", (193, 134, 46)],
    "['Enojado']" : ["Enojado", (43, 57, 192)],
    "['Sorprendido']" : ["Sorprendido", (99, 180, 40)]
}

## ▪️ Prueba de la red neuronal entrenada para reconocer expresiones:

In [18]:
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Inicializar la cámara.
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error al abrir la cámara.")
    exit()
    
# Crear un objeto CascadeClassifier para la detección de rostros.
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Verificar si la cámara se abrió correctamente
with mp_face_mesh.FaceMesh(static_image_mode=False,
                           max_num_faces=1,
                           min_detection_confidence=0.5) as face_mesh:
    while True:
        # Capturar un solo fotograma
        ret, frame = cap.read()

        # Verificar si la lectura del fotograma fue exitosa
        if not ret:
            print("Error al leer el fotograma.")
            break

        # Capturar un solo fotograma.
        ret, frame = cap.read()

        # Verificar si la lectura del fotograma fue exitosa
        if not ret:
            print("Error al leer el fotograma.")
            break

        # Transformar el frame en escala de grises para una mejor detección.
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Realizar la detección de rostros.
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:

            # Recortar y guardar el rectángulo del rostro
            face_roi = frame[y:y+h, x:x+w]

            # Se aplica la mascara de punto facemesh.
            frame_rgb = cv2.cvtColor(face_roi, cv2.COLOR_BGR2RGB)
            results = face_mesh.process(frame_rgb)

            if results.multi_face_landmarks is not None:
                for face_landmarks in results.multi_face_landmarks:

                    #>>> PUNTOS DE BOCA >>>>>>>>>
                    # Punto medio labio superior.
                    p13 = face_landmarks.landmark[13]
                    # Punto medio labio inferior.
                    p14 = face_landmarks.landmark[14]
                    # Punto extremo izquierdo.
                    p61 = face_landmarks.landmark[61]
                    # Punto extremo derecho.
                    p306 = face_landmarks.landmark[306]

                    #>>> PUNTOS DE CEJA >>>>>>>>>
                    # Ceja izquierda.
                    p65 = face_landmarks.landmark[65]
                    # Ceja derecha.
                    p295 = face_landmarks.landmark[295]

                    #>>> PUNTOS DE OJO >>>>>>>>>
                    # Ojo izquierdo ----
                    # Punto superior.
                    p159 = face_landmarks.landmark[159]
                    # Punto inferior.
                    p145 = face_landmarks.landmark[145]
                    # Ojo derecho ------
                    # Punto superior.
                    p386 = face_landmarks.landmark[386]
                    # Punto inferior.
                    p374 = face_landmarks.landmark[374]

                    # === Se hacen los respectivos cálculos de distancias entre puntos.
                    # Distancia entre Ceja y Ojo.
                    d1 = (calcular_distancia(p65, p159) + calcular_distancia(p295, p386)) / 2
                    # Apertura de Ojo".
                    d2 = (calcular_distancia(p159, p145) + calcular_distancia(p386, p374)) / 2
                    # Ancho de Boca.
                    d3 = calcular_distancia(p61, p306)
                    # Apertura de Boca.
                    d4 = calcular_distancia(p13, p14)
                    # Distancia de Extremos y Superior Boca.
                    d5 = (calcular_distancia(p61, p13) + calcular_distancia(p306, p13)) / 2
                    # Distancia de Extremos e Inferior Boca.
                    d6 = (calcular_distancia(p61, p14) + calcular_distancia(p306, p14)) / 2
                    # Clasificador.
                    d7 = expresion

                    # Se reaiza la predicción.
                    prediccion = instancia_red.predict(np.array([d1, d2, d3, d4, d5, d6]).reshape(1, -1))
                    pred = str(prediccion)
                    
                    # Se escribe la predicción en el fotograma.
                    cv2.putText(frame, "Expresion:" , (x, y-40), cv2.FONT_HERSHEY_PLAIN , 0.8, (255, 255, 255), 1)
                    cv2.putText(frame, expresion[pred][0], (x, y-10), cv2.FONT_HERSHEY_DUPLEX , 1, expresion[pred][1], 2)

        # Mostrar el fotograma con los rostros detectados
        cv2.imshow('Sistema de Reconocimiento de Exrpesiones', frame)

        # Salir del bucle si se presiona la tecla 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Liberar la cámara y cerrar la ventana
cap.release()
cv2.destroyAllWindows()
print(">>> Ejecución exitosa.")

>>> Ejecución exitosa.
